# library loading

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import csv
import time
import re # 정규표현식
import pandas as pd #csv저장

# information
- 상품명(pname), 총별점(tstar), 닉네임(name), 별점(star), 작성일(pdate), 기타리뷰(rv)
- pname_l , ..., non_l(리뷰없음), fail_l(경로가 다른지 안가져와진 애들)

In [2]:
# 화면 주소 정의
## 스킨케어
a = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010001'
## 마스크팩
b = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010009&gateCd=Drawer'
## 클렌징
c = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010010&gateCd=Drawer'
## 선케어
d = 'https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo=10000010011&gateCd=Drawer'

# self function

In [3]:
def rv_num2():
    # 리뷰 수 //*[@id="gdasContentsArea"]/div/div[1]/div/div[2]/p[1]/em
    global q
    global pn
    global pc
    q = driver.find_element(By.XPATH, '//*[@id="reviewInfo"]/a/span').text[1:-1]
    q = int(re.sub(r"[^0-9]", "", q))
    if q == 0:
        
        pn = 'zero'
        pc = 'zero'
    elif 0 < q <= 10:
        pn = 'zero'
        pc = q
    else:   
    # page_num(리뷰 페이지 수), product_count(마지막페이지의 리뷰수)
        pn = (q // 10) + 1 
        pc = q % 10

In [4]:
def rv_info2():
    try :
        global pname
        # //*[@id="Contents"]/div[2]/div[2]/div/p[2]
        # //*[@id="Contents"]/div[2]/div[2]/div/p[2]
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
        global tstar
        # #gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong
        tstar = driver.find_element(By.CSS_SELECTOR,'#gdasContentsArea > div > div.product_rating_area.review-write-delete > div > div.star_area > p.num > strong').text
    except:
        print('별점은없어')
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text

In [5]:
def list_up(n=10):  # 파라미터는 한 페이지 안에 있는 리뷰수
    for i in range(1, n+1):
        pname_l.append(pname)
        tstar_l.append(tstar)
        # 닉네임
        # //*[@id="gdasList"]/li[1]/div[1]
        # //*[@id="gdasList"]/li[2]/div[1]/div
        # //*[@id="gdasList"]/li[2]/div[1]/div/p/a
        y = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]').text
        name = y.split(sep='\n')[0]
        name_l.append(name)
        time.sleep(0.5) # x에서 계속 멈춰서 1일떈 끝까지 감
        # 리뷰 블럭 //*[@id="gdasList"]/li[4]/div[2]
        x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{i}]/div[2]').text
        rv = x.split(sep='\n')
        ## 별점
        star = rv[0]
        star_l.append(star)
        ## 작성일
        pdate = rv[1]
        pdate_l.append(pdate)
        ##기타
        del rv[0:2] 
        del rv[-2:]
        rv_l.append(rv)
        print(f'_{i}') 

In [6]:
def one_product2(pn, pc):
    list_up()
    for l in range(2, pn+1): 
        if str(l)[-1] == '0':    
            try: # 10페이지가 마지막인 경우
                driver.find_element(By.LINK_TEXT, f'{l}').click()
                driver.implicitly_wait(3)
                # 10개의 정보 가져오는 함수 기본함수정의해서파라미터로 숫자넣기(10) 
                print(l)
                list_up()
                driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
                driver.implicitly_wait(3)
                print(l)
                list_up()
            except:
                driver.find_element(By.LINK_TEXT, f'{l}').click()
                driver.implicitly_wait(3)
                # 10개의 정보 가져오는 함수 기본함수정의해서파라미터로 숫자넣기(10) 
                list_up(pc)
                print(l, 'complete')
                
            
            # 함수가져오기 list_up()
        elif str(l)[-1] == '0' and l == pn: # 다음페이지가 마지막인 경우
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            list_up()
            driver.find_element(By.PARTIAL_LINK_TEXT, f'다음').click()
            driver.implicitly_wait(3)
            print(l)
            if pc != 0: # pc가 0이면 for문 안돌아가서 
                list_up(pc)
                print('수집완료^^')
            else:
                list_up()
                print('수집완료^')
        elif str(l)[-1] == '1':
            print(l)
        elif l == (pn): # 마지막 페이지
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            if pc != 0:
                list_up(pc)
                print('수집완료^^')
            else:
                list_up(pc)
                print('수집완??')
            # 끝자리수만큼 포문 pc list_up(pc)
            print(l)
        else:
            driver.find_element(By.LINK_TEXT, f'{l}').click()
            driver.implicitly_wait(3)
            list_up()
            print(l)

In [7]:
def try_pro():
    try:
        rv_num2()
        rv_info2()
        if pn == 'zero' and pc == 'zero':
            # 상품리뷰 없음. 
            non_l.append(pname) # 상품명 리스트에는 안올라감
        elif pn == 'zero':
            list_up(pc)
            print('출력완료^^')
        else:
            one_product2(pn, pc)
    except:
        pname = driver.find_element(By.XPATH,'//*[@id="Contents"]/div[2]/div[2]/div/p[2]').text
        fail_l.append(pname)
        print("==== 문제 발생 ====", pname)

## do it

In [3]:
#  driver setting
URL = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' # 첫화면
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

# chromedriver path
sv = Service(executable_path= 'C:\\Users\\admin\\Downloads\\chroemdriver')
driver = webdriver.Chrome(service=sv, options=ops)

In [4]:
# page moving
driver.get(url=a)
driver.implicitly_wait(3)
# 기본페이지 이동
# 스킨케어_토너 페이지로 이동
driver.find_element(By.XPATH, f'//*[@id="Contents"]/div/div[1]/ul/li[1]/a').click()
driver.implicitly_wait(3)
# 리뷰 48개로 선택
driver.find_element(By.XPATH, '//*[@id="Contents"]/div[5]/div[2]/ul/li[3]/a').click()
driver.implicitly_wait(3)

In [13]:
##### 연습 리뷰 텍스트만 가져오기
x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{4}]/div[2]').text
rv = x.split(sep='\n')
del rv[-2:]
del rv[:2]
rv[-3:], rv[:2]

(['총평: 보습은 막 엄청나진 않았지만 적당했고 끈적이지 않아 악건성이 아니라면 호불호가 갈리지 않을듯하고 여드름은 제가 여드름이 잘 나지않아 추천해드리긴 조심스럽지만 세일할때 구매하면 가성비도 있고, 무난해서 너무 보습력이 강하지도 건조하지도 않는 무난한 제품 찾는다면 사용하기 괜찮을거 같습니다.',
  '',
  '도움이 되었다면 도움 눌러주세요! 감사합니다.'],
 ['피부타입', '건성에 좋아요'])

In [16]:
## 키워드도 try구문 써볼까?
try:
    x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[5]/div[2]/div[2]').text
    print(x)
except:
    print('there is no keyword')
    


피부타입
복합성에 좋아요
피부고민
진정에 좋아요
자극도
자극없이 순해요


In [45]:
x = driver.find_element(By.XPATH, f'//*[@id="gdasList"]/li[{7}]/div[2]/div[1]/span[1]/span').text
x =list(x)
# del x[0]
# x = list(x)
int(x[-2])
# find_elements는 text 옵션이 없음.

5

In [37]:
#### 리뷰 앞문장과 마지막 문장 가져오기
try:
    x = driver.find_element(By.XPATH, '//*[@id="gdasList"]/li[7]/div[2]/div[3]').text
    # txt inner
    rv = x.split(sep='\n')
    if len(rv) == 1:
        text = rv[0]
    else:
        text = rv[0]+ rv[-1]
    print(text)
except:
    try:  # 리뷰 아우터만 존재하는 경우
        x = driver.find_element(By.XPATH, '//*[@id="gdasList"]/li[7]/div[2]/p').text
    except:
        x = 'no text'  # 해당 리뷰 구조가 없으면 리뷰가 없는걸로
    print(x)

no text


In [53]:
# 그 페이지의 리뷰 수 
a = driver.find_elements(By.CLASS_NAME, 'review_cont')
len(a)
#gdasList > li:nth-child(1) > div.review_cont

10

In [10]:
# 리스트 정의
pname_l = []
tstar_l = []
name_l = []
star_l = []
pdate_l = []
rv_l = []

non_l = []
fail_l = []

In [11]:
# crawler 
# for문 가능?
for x in range(2,11):
    try:
        for i in range(1, 13):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH ,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                time.sleep(3)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                time.sleep(3)
                try_pro()
                driver.back()
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.find_element(By.LINK_TEXT, f'{10+x}').click()
        driver.implicitly_wait(3)
    except:
        continue

_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
2
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
3
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
4
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
5
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
6
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
7
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
8
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
9
10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
11
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
12
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
13
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
14
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
15
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
16
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
17
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
18
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
19
20
==== 문제 발생 ==== [2022어워즈]아누아 어성초 77 토너 어워즈 한정 기획 (500ml+250ml)
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
2
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
3
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
4
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
5
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
6
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
7
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
8
_1
_2
_3
_4
_5
_6
_7
_8
_9
_10
9
1

In [12]:
# next button
driver.find_element(By.PARTIAL_LINK_TEXT, '다음').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"다음"}
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x01206643]
	(No symbol) [0x0119BE21]
	(No symbol) [0x0109DA9D]
	(No symbol) [0x010D1342]
	(No symbol) [0x010D147B]
	(No symbol) [0x01108DC2]
	(No symbol) [0x010EFDC4]
	(No symbol) [0x01106B09]
	(No symbol) [0x010EFB76]
	(No symbol) [0x010C49C1]
	(No symbol) [0x010C5E5D]
	GetHandleVerifier [0x0147A142+2497106]
	GetHandleVerifier [0x014A85D3+2686691]
	GetHandleVerifier [0x014ABB9C+2700460]
	GetHandleVerifier [0x012B3B10+635936]
	(No symbol) [0x011A4A1F]
	(No symbol) [0x011AA418]
	(No symbol) [0x011AA505]
	(No symbol) [0x011B508B]
	BaseThreadInitThunk [0x776400F9+25]
	RtlGetAppContainerNamedObjectPath [0x779E7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x779E7B8E+238]


In [ ]:
# crawler 
# for문 가능?
for x in range(2,7):
    try:
        for i in range(1, 13):
            for j in range(1,5):
                # 상품 선택
                driver.find_element(By.XPATH,f'//*[@id="Contents"]/ul[{i+1}]/li[{j}]/div/a/img').click()
                time.sleep(3)
                # 리뷰칸 선택
                driver.find_element(By.XPATH,f'//*[@id="reviewInfo"]/a').click()
                time.sleep(3)
                try_pro()
                driver.back()
    except:
        print('어디서 오류가 났으려나? 일단 pass^^')
    try:
        driver.find_element(By.LINK_TEXT, f'{10+x}').click()
        driver.implicitly_wait(3)
    except:
        continue

In [13]:
# restore data
try:   
    rv_rv = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l, 'name':name_l,'star':star_l,'pdate':pdate_l, 'rv':rv_l})
    rv_rv.to_csv('./olive_rv_datasets.csv', sep = ',', encoding = 'utf-8-sig')
    print('Yes')
except:
    rv_rv = pd.DataFrame({'star':star_l,'pdate':pdate_l, 'rv':rv_l})
    rv_rv.to_csv('./olive_rv_datasets.csv', sep = ',', encoding = 'utf-8-sig')
    rv_name = pd.DataFrame({'pname':pname_l, 'tstar':tstar_l})
    rv_name.to_csv('./olive_info_datasets.csv', sep = ',', encoding = 'utf-8-sig')
    print('this')

this


In [14]:
rv_zero = pd.DataFrame({'pname':non_l})
rv_zero.to_csv('./zero_data.csv', sep = ',', encoding = 'utf-8-sig')
rv_fail = pd.DataFrame({'pname':fail_l})
rv_fail.to_csv('./fail_data.csv', sep = ',', encoding = 'utf-8-sig')

In [ ]:
# exit
driver.close()

In [15]:
from datetime import datetime

current_time = datetime.now()
print(current_time)

2023-01-21 11:26:18.804263
